In [1]:
import pandas as pd
import re
from transformers import T5EncoderModel, T5Tokenizer
import torch

In [2]:
k = torch.load('./data/davis/processed/train/processed_data_train_0.pt')
k

Data(x=[25, 22], edge_index=[2, 54], edge_attr=[54, 6], y=[1], target=[1, 1200], prot5_embedding=[1, 1200, 1024])

In [7]:
df = pd.read_csv('./data/davis/raw/data_train.csv')
df.shape

(25046, 3)

In [8]:
df.head(10)

,compound_iso_smiles,target_sequence,affinity
0,O=C(NC1CCNCC1)c1[nH]ncc1NC(=O)c1c(Cl)cccc1Cl,MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAG...,5.000000
1,CSc1cccc(Nc2ncc3cc(-c4c(Cl)cccc4Cl)c(=O)n(C)c3...,MLRGGRRGQLGWHSWAAGPGSLLAWLILASAGAAPCPDACCPHGSS...,5.000000
2,COc1cc2ncnc(Nc3ccc(F)c(Cl)c3)c2cc1OCCCN1CCOCC1,MGPAPLPLLLGLFLPALWRRAITEAREEAKPYPLFPGPFPGSLQTD...,5.000000
3,COC1C(N(C)C(=O)c2ccccc2)CC2OC1(C)n1c3ccccc3c3c...,MFRKKKKKRPEISAPQNFQHRVHTSFDPKEGKFVGLPPQWQNILDT...,5.000000
4,CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc...,TMPPRPSSGELWGIHLMPPRILVECLLPNGMIVTLECLREATLITI...,5.000000
5,CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc...,MDLSMKKFAVRRFFSVYLRRKSRSKSSSLSRLEEEGVVKEIDISHH...,5.000000
6,CCN1CCN(Cc2ccc(NC(=O)Nc3ccc(Oc4cc(NC)ncn4)cc3)...,MSSCVSSQPSSNRAAPQDELGGRGSSSSESQKPCEALRGLSSLSIH...,6.657577
7,Oc1cccc(-c2nc(N3CCOCC3)c3oc4ncccc4c3n2)c1,MELENIVANSLLLKARQGGYGKKSGRSKKWKEILTLPPVSQCSELR...,5.000000
8,CCn1c(-c2nonc2N)nc2c(C#CC(C)(C)O)ncc(OCC3CCCNC...,MEGGPAVCCQDPRAELVERVAAIDVTHLEEADGGPEPTRNGVDPPP...,5.000000
9,CN(C)CC=CC(=O)Nc1cc2c(Nc3ccc(F)c(Cl)c3)ncnc2cc...,MGSNKSKPKDASQRRRSLEPAENVHGAGGGAFPASQTPSKPASADG...,5.552842


In [2]:
def get_t5_model(device):
    model = T5EncoderModel.from_pretrained("../protT5/protT5_checkpoint/", torch_dtype=torch.float16)
    model = model.to(device) # move model to GPU
    model = model.eval() # set model to evaluation model
    tokenizer = T5Tokenizer.from_pretrained("Rostlab/prot_t5_xl_uniref50", do_lower_case=False ) 

    return model, tokenizer

In [14]:
target_len = 1200
device = 'cuda:0'

In [5]:
t5_model, t5_tokenizer = get_t5_model(device=device)

In [15]:
for idx, row in df.iterrows():
    sequence = row['target_sequence']
    print(sequence)
    reformed_seq = sequence[:target_len]
    prot5_seq = " ".join(list(re.sub(r"[UZOB]", "X", reformed_seq)))
    ids = t5_tokenizer.batch_encode_plus(prot5_seq, add_special_tokens=True, max_length=target_len, padding='max_length')
    input_ids = torch.tensor(ids['input_ids']).to(device)
    attention_mask = torch.tensor(ids['attention_mask']).to(device)
    with torch.no_grad():
        embedding = t5_model(input_ids=input_ids,attention_mask=attention_mask)

    print(f'Shape of embedding is {embedding.shape}')
    if idx == 1:
        break

MVSYWDTGVLLCALLSCLLLTGSSSGSKLKDPELSLKGTQHIMQAGQTLHLQCRGEAAHKWSLPEMVSKESERLSITKSACGRNGKQFCSTLTLNTAQANHTGFYSCKYLAVPTSKKKETESAIYIFISDTGRPFVEMYSEIPEIIHMTEGRELVIPCRVTSPNITVTLKKFPLDTLIPDGKRIIWDSRKGFIISNATYKEIGLLTCEATVNGHLYKTNYLTHRQTNTIIDVQISTPRPVKLLRGHTLVLNCTATTPLNTRVQMTWSYPDEKNKRASVRRRIDQSNSHANIFYSVLTIDKMQNKDKGLYTCRVRSGPSFKSVNTSVHIYDKAFITVKHRKQQVLETVAGKRSYRLSMKVKAFPSPEVVWLKDGLPATEKSARYLTRGYSLIIKDVTEEDAGNYTILLSIKQSNVFKNLTATLIVNVKPQIYEKAVSSFPDPALYPLGSRQILTCTAYGIPQPTIKWFWHPCNHNHSEARCDFCSNNEESFILDADSNMGNRIESITQRMAIIEGKNKMASTLVVADSRISGIYICIASNKVGTVGRNISFYITDVPNGFHVNLEKMPTEGEDLKLSCTVNKFLYRDVTWILLRTVNNRTMHYSISKQKMAITKEHSITLNLTIMNVSLQDSGTYACRARNVYTGEEILQKKEITIRDQEAPYLLRNLSDHTVAISSSTTLDCHANGVPEPQITWFKNNHKIQQEPGIILGPGSSTLFIERVTEEDEGVYHCKATNQKGSVESSAYLTVQGTSDKSNLELITLTCTCVAATLFWLLLTLFIRKMKRSSSEIKTDYLSIIMDPDEVPLDEQCERLPYDASKWEFARERLKLGKSLGRGAFGKVVQASAFGIKKSPTCRTVAVKMLKEGATASEYKALMTELKILTHIGHHLNVVNLLGACTKQGGPLMVIVEYCKYGNLSNYLKSKRDLFFLNKDAALHMEPKKEKMEPGLEQGKKPRLDSVTSSESFASSGFQEDKSLSDVEEEEDSDGFYKEPITMEDLI

OutOfMemoryError: CUDA out of memory. Tried to allocate 5.49 GiB (GPU 0; 7.79 GiB total capacity; 4.07 GiB already allocated; 799.44 MiB free; 6.11 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [6]:
import torch

t1 = torch.load("data/davis/raw/train/0.pt")
t2 = torch.load("data/davis/raw/train/1.pt")

In [7]:
t1.shape, t2.shape

(torch.Size([1, 1200, 480]), torch.Size([1, 1200, 480]))

In [5]:
t1

tensor([[[ 0.1431,  0.1823,  0.2212,  ...,  0.4056,  0.0531, -0.4450],
         [ 0.2954,  0.4748, -0.1557,  ...,  0.2240,  0.0875, -0.1658],
         [ 0.2894,  0.0873,  0.2931,  ...,  0.4202,  0.2012,  0.0186],
         ...,
         [-0.0297, -0.1347, -0.1772,  ...,  0.4727,  0.0842, -0.0427],
         [-0.0548, -0.1386, -0.1192,  ...,  0.6624,  0.0184, -0.1239],
         [-0.1602, -0.1580, -0.2965,  ...,  0.6188, -0.2189,  0.0858]]],
       requires_grad=True)

In [11]:
import torch

# Example input tensor and node count vector
input_tensor = torch.randint(0, 15, (10, 8))  # Shape: (total nodes in batch, feature diin
print(input_tensor)
print(input_tensor.shape)
node_count = torch.tensor([0, 0, 0, 1, 1, 1, 1, 2, 2, 3])  # Number of nodes in each 
print(node_count.shape)

from torch_geometric.utils import to_dense_batch

out, mask = to_dense_batch(input_tensor, node_count)
out


tensor([[ 9, 11, 13,  9,  7,  4,  9,  9],
        [14, 12, 13, 11, 12,  2,  1,  5],
        [ 5,  5,  3, 10, 12,  8, 10, 13],
        [12, 13, 10,  7,  6, 12,  6,  4],
        [ 0,  2,  4, 12,  6, 11, 12,  4],
        [ 5,  3,  8,  6,  9, 13, 13,  9],
        [ 0,  4,  8,  9,  0,  3, 13, 12],
        [10,  7,  1, 14,  8, 10,  3,  8],
        [ 1, 10, 11,  9,  8,  6,  1,  3],
        [ 7,  7,  3,  3,  5,  7, 10,  8]])
torch.Size([10, 8])
torch.Size([10])


tensor([[[ 9, 11, 13,  9,  7,  4,  9,  9],
         [14, 12, 13, 11, 12,  2,  1,  5],
         [ 5,  5,  3, 10, 12,  8, 10, 13],
         [ 0,  0,  0,  0,  0,  0,  0,  0]],

        [[12, 13, 10,  7,  6, 12,  6,  4],
         [ 0,  2,  4, 12,  6, 11, 12,  4],
         [ 5,  3,  8,  6,  9, 13, 13,  9],
         [ 0,  4,  8,  9,  0,  3, 13, 12]],

        [[10,  7,  1, 14,  8, 10,  3,  8],
         [ 1, 10, 11,  9,  8,  6,  1,  3],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0]],

        [[ 7,  7,  3,  3,  5,  7, 10,  8],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0],
         [ 0,  0,  0,  0,  0,  0,  0,  0]]])

In [15]:
t = torch.randn((4, 16, 32))
t1 = torch.nn.AdaptiveAvgPool1d((1, 32))
print(t1(t).shape)

RuntimeError: adaptive_avg_pool1d() argument 'output_size' should contain one int (got 2)